Dependencies

In [135]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re

data = pd.read_csv('diffusion_db_unaltered.csv')
nsfw_words = pd.read_csv('nsfw_words.csv')
sfw_words = pd.read_csv('sfw_words.csv')

This cell handles preliminary variables such as p(NSFW) and p(SFW) which determine the probability a prompt is nsfw or sfw using the image_nsfw score

In [ ]:
prompt_count = len(data)
nsfw_count = len(data[data['image_nsfw'] >= .5])
sfw_count = len(data[data['image_nsfw'] < .5])

p_nsfw = nsfw_count / prompt_count
p_sfw = sfw_count / prompt_count

379
91189
0.0850015
0.9149985


In [ ]:
commonwords = [
    "the", "a", "an", "is", "are", "and", "to", "in", "that", "have", "has", 
    "with", "this", "these", "those", "it", "for", "on", "be", "was", "were", 
    "can", "will", "would", "should", "may", "might", "do", "does", "did", "of", "its", "their"
]

In [133]:
def cleanPrompt(prompt):
    prompt_list = prompt.lower().split(' ')
    prompt_list = [word for word in prompt_list if word not in commonwords]
    prompt = ' '.join(prompt_list)
    return re.sub(r'[^a-zA-Z0-9 ]', '', prompt)



def classifyMessage(message, alpha):
    safe = p_sfw
    unsafe = p_nsfw
    
    # This sections gets the p(word|N) and p(word|S) for each word in the message.
    word_probs_nsfw = {}
    word_probs_sfw = {}

    for word in message.split():
        if word not in word_probs_nsfw.keys():
            word_probs_nsfw[word] = [alpha, 1] # First index is the probability, the second index is the amount of times that the word appears in the prompt
            word_probs_sfw[word] = [alpha, 1]

            # check if word is in nsfw_words dataframe
            if word in nsfw_words.values:
                word_probs_nsfw[word] = [(nsfw_words[nsfw_words['word'] == word]['count'].values[0]) / len(nsfw_words), 1]
            else:
                word_probs_nsfw[word] = [alpha / len(nsfw_words), 1]

            if word in sfw_words.values:
                word_probs_sfw[word] = [(sfw_words[sfw_words['word'] == word]['count'].values[0])  / len(sfw_words), 1]
            else:
                word_probs_sfw[word] = [alpha / len(sfw_words), 1]
        else:
            word_probs_nsfw[word][1]+=1
            word_probs_sfw[word][1]+=1

    for word in word_probs_nsfw.keys():
        # multiply safe and unsafe probabilities by the word probabilities but each probability is to the exponent of the amount of occurances
        safe *= word_probs_sfw[word][0] ** word_probs_sfw[word][1]
        unsafe *= word_probs_nsfw[word][0] ** word_probs_nsfw[word][1]

    # for word, score in word_probs_nsfw.items():
    #     print(f"{word}: {score[0]}")

    # print("\nSAFE FOR WORK WORDS:")

    # for word, score in word_probs_sfw.items():
    #     print(f"{word}: {score[0]}")

    if safe > unsafe:
        return "SFW"
    else:
        return "NSFW"



In [129]:
examples = pd.read_csv('examples.csv')['Prompt']

results = {
    "Prompt": [],
    "Results": []
}

for example in examples:
    result = classifyMessage(cleanPrompt(example), 1)
    results['Prompt'].append(example)
    if result == "SFW":
        results['Results'].append(1)
    else:
        results['Results'].append(0)

results = pd.DataFrame(results)
results.to_csv('results.csv', index=False)

In [138]:
import csv
import random

# Safe for Work (SFW) prompts
sfw_prompts = [
    "A peaceful beach at sunset with gentle waves", "A child flying a kite on a windy day", "A colorful hot air balloon floating over a mountain range",
    "A cozy living room with a crackling fireplace", "A person riding a bicycle through a city park", "A beautiful sunrise over the ocean horizon",
    "A scenic view of a forest trail covered in autumn leaves", "A cute cat sleeping peacefully on a soft couch", "A family having a picnic in a green meadow",
    "A large, bright sunflower field in full bloom", "A dog happily running through a field of daisies", "A young couple walking hand-in-hand down a beach",
    "A peaceful lake with a rowboat floating in the middle", "A mountain cabin surrounded by snow in winter", "A group of friends gathered around a campfire",
    "A person playing the piano in a dimly lit room", "A woman practicing yoga in a serene garden", "A mother reading a bedtime story to her child",
    "A rainbow stretching across the sky after a storm", "A squirrel gathering acorns in a forest", "A fresh cup of coffee sitting on a kitchen table",
    "A child playing with a toy train set", "A man riding a horse across a grassy field", "A person taking a photograph of a beautiful flower",
    "A bright autumn day with leaves falling from the trees", "A cozy winter night with a blanket and hot cocoa", "A couple stargazing on a clear night",
    "A close-up of a red rose in full bloom", "A person playing chess in the park", "A grandmother knitting a scarf in her rocking chair",
    "A group of people volunteering at a food bank", "A man painting a picture on a canvas", "A woman dancing in a sunflower field",
    "A quiet street lined with old brick houses", "A baby smiling in a high chair", "A peaceful town square with a fountain", "A person walking a dog through the neighborhood",
    "A bicycle leaning against a tree in a park", "A farmer harvesting crops on a bright morning", "A little girl riding her scooter around the block",
    "A scenic waterfall in a lush jungle", "A boy flying a paper plane through a field", "A woman baking cookies in the kitchen", "A group of kids playing soccer in the park"
]

# Not Safe for Work (NSFW) prompts for Image Generation (Violence, sexual content, mature themes, etc.)
nsfw_prompts = [
    "A chaotic street brawl with people throwing punches", "A person in a fight with blood splattering around", "A provocative pin-up model posing on a bed", 
    "A dark alley with shady characters exchanging goods", "A masked figure holding a weapon in a threatening stance", "A gruesome crime scene with bloodied objects", 
    "A seductive figure in a red dress with an intense look", "A person bound with ropes in a dark room", "A close-up of a knife with blood dripping off of it", 
    "A violent car crash with fire and smoke", "A woman tied up in chains, with a distressed look", "A provocative dancer in a dimly lit club", 
    "A shattered window with someone escaping", "A dramatic scene of a person running from an explosion", "A stormy night with thunder and lightning striking a building", 
    "A person threatening another with a gun", "An ominous figure standing in a dark corner of a room", "A person with a blood-stained shirt holding a baseball bat", 
    "A night club scene with flashing lights and people dancing provocatively", "A dark figure standing with a silhouette of a weapon", "A passionate kiss between two people in an intimate setting",
    "A car chase through a city with explosions in the background", "A broken doll lying in an abandoned house", "A bruised woman in a hospital bed with tears in her eyes", 
    "A pair of handcuffs on a wooden chair in a dark room", "A scene from a horror movie with a ghostly figure", "A pair of eyes peeking through the blinds in the dark",
    "A man being chased by a monster in a dark alley", "A zombie with blood dripping from its mouth", "A hand reaching out of a grave in a dark cemetery", 
    "A woman with a fierce expression holding a whip", "A dark scene of a battle between superheroes", "A blood-soaked floor in a crime scene", 
    "A creepy figure with glowing red eyes standing in the distance", "A silhouette of a person holding a chainsaw in a field", "A dark figure lurking in an abandoned house",
    "A person sneaking through a forest with a knife in hand", "A biker gang standing around their motorcycles in a dark parking lot", "A vampire with sharp fangs staring into the camera",
    "A woman in a leather outfit holding a whip in a BDSM scene", "A violent gang fight in an alley with people holding weapons", "A seductive couple in an intimate, dark room", 
    "A person wearing a mask and holding a bloody machete", "A haunted mansion with windows shattered by force", "A close-up of a hand holding a gun pointed directly at the camera",
    "A bloody handprint on the wall of an abandoned house", "A violent storm tearing apart a city skyline", "A person covered in tattoos in a darkened alley", 
    "A demon with fiery red eyes and smoke surrounding it", "A close-up of a man’s face showing rage with clenched fists", "A dramatic scene of a person holding a broken mirror in their hand"
]

# Combine both lists and label them
prompts = []
for prompt in sfw_prompts:
    prompts.append([prompt, "SFW"])
for prompt in nsfw_prompts:
    prompts.append([prompt, "NSFW"])

# Shuffle the list to randomize order
random.shuffle(prompts)

# Write to CSV
with open('image_generator_prompts.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Prompt", "Label"])  # Write the header
    writer.writerows(prompts)  # Write the data

print("CSV file 'image_generator_prompts.csv' created successfully.")


CSV file 'image_generator_prompts.csv' created successfully.


In [137]:
prompts = pd.read_csv('prompts.csv')

# Assuming you have a dataset with actual labels for comparison:
# X_test - List of messages (prompts)
# y_test - True labels for the messages (either 'SFW' or 'NSFW')

X_test = prompts['Prompt'].tolist()  # List of prompts (messages)
y_test = prompts['Label'].tolist()   # List of true labels (SFW or NSFW)

# Predict the results using your Naïve Bayes model
y_pred = [classifyMessage(cleanPrompt(message), alpha=1.0) for message in X_test]

# Evaluate performance:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='NSFW')  # Positive class = 'NSFW'
recall = recall_score(y_test, y_pred, pos_label='NSFW')
f1 = f1_score(y_test, y_pred, pos_label='NSFW')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.8181818181818182
Precision: 0.8055555555555556
Recall: 0.8529411764705882
F1 Score: 0.8285714285714286
